# Práctica de Laboratorio en NLP - Tópicos
Tema: Extracción de tópicos

Introducción:

En esta práctica, trabajaremos en un proyecto de análisis de reclamaciones de un restaurante. La empresa desea saber de qué se quejan más los comensales, para identificar áreas de mejora. 

## 1. Desarrolla la extracción de tópicos con LDA

## 2. Pruébalo en la interfaz de ChatGPT y pídele los topics. ¿Cambian mucho respecto al LDA? Comenta las diferencias.


In [1]:
### Importación de librerías

import spacy
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

nlp = spacy.load("es_core_news_lg")

In [2]:
sentencias = [
    "La comida llegó fría y tardó mucho en ser servida.",
    "El servicio fue lento y el personal parecía desinteresado.",
    "Pedimos un plato vegetariano y nos trajeron uno con carne.",
    "Las mesas estaban sucias y no habían sido limpiadas correctamente.",
    "No había suficiente iluminación en el área de comedor.",
    "El restaurante estaba abarrotado y no se respetaron las reservas.",
    "La música estaba demasiado alta y no se podía mantener una conversación.",
    "La bebida que pedimos nunca llegó a la mesa.",
    "Los platos estaban mal sazonados y carecían de sabor.",
    "El menú tenía errores de ortografía y gramática.",
    "El restaurante tenía un olor desagradable.",
    "No había opciones vegetarianas en el menú.",
    "Se nos cobró de más en la factura.",
    "El baño del restaurante estaba sucio y sin papel higiénico.",
    "El vino que pedimos estaba agrio y parecía estar mal almacenado.",
    "La carne de mi plato estaba cruda en el centro.",
    "Las sillas eran incómodas y difíciles de mover.",
    "El personal fue grosero y poco atento.",
    "El restaurante estaba demasiado caliente y sin aire acondicionado.",
    "No se nos proporcionaron utensilios de mesa ni servilletas.",
    "El postre que pedimos estaba rancio y no era comestible.",
    "El menú era limitado y no ofrecía opciones para alergias alimentarias.",
    "El restaurante tenía una plaga de insectos.",
    "Los platos se veían descuidados y mal presentados.",
    "La factura tenía errores en los cálculos de los precios."
]




In [3]:
import re
import nltk
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('spanish')


def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

norm_corpus=[]

for document in sentencias:
    norm_corpus.append(normalize_document(document))

norm_corpus


['comida llegó fría tardó ser servida .',
 'servicio lento personal parecía desinteresado .',
 'pedimos plato vegetariano trajeron carne .',
 'mesas sucias sido limpiadas correctamente .',
 'suficiente iluminación área comedor .',
 'restaurante abarrotado respetaron reservas .',
 'música demasiado alta podía mantener conversación .',
 'bebida pedimos nunca llegó mesa .',
 'platos mal sazonados carecían sabor .',
 'menú errores ortografía gramática .',
 'restaurante olor desagradable .',
 'opciones vegetarianas menú .',
 'cobró factura .',
 'baño restaurante sucio papel higiénico .',
 'vino pedimos agrio parecía mal almacenado .',
 'carne plato cruda centro .',
 'sillas incómodas difíciles mover .',
 'personal grosero atento .',
 'restaurante demasiado caliente aire acondicionado .',
 'proporcionaron utensilios mesa servilletas .',
 'postre pedimos rancio comestible .',
 'menú limitado ofrecía opciones alergias alimentarias .',
 'restaurante plaga insectos .',
 'platos veían descuidados

In [4]:
### Creación del vectorizador y generación de la matriz Tf-idf

tfifd_vec = TfidfVectorizer()

TFIDF = tfifd_vec.fit_transform(norm_corpus)

TFIDF

<25x88 sparse matrix of type '<class 'numpy.float64'>'
	with 110 stored elements in Compressed Sparse Row format>

In [5]:
# Visualización de la matriz

# Obtenemos el vocabulario para poner las etiquetas de las columnaas
vocab = tfifd_vec.get_feature_names_out()

print("Palabras en el vocabulario: ", len(vocab))
print(vocab)

Palabras en el vocabulario:  88
['abarrotado' 'acondicionado' 'agrio' 'aire' 'alergias' 'alimentarias'
 'almacenado' 'alta' 'atento' 'baño' 'bebida' 'caliente' 'carecían'
 'carne' 'centro' 'cobró' 'comedor' 'comestible' 'comida' 'conversación'
 'correctamente' 'cruda' 'cálculos' 'demasiado' 'desagradable'
 'descuidados' 'desinteresado' 'difíciles' 'errores' 'factura' 'fría'
 'gramática' 'grosero' 'higiénico' 'iluminación' 'incómodas' 'insectos'
 'lento' 'limitado' 'limpiadas' 'llegó' 'mal' 'mantener' 'menú' 'mesa'
 'mesas' 'mover' 'música' 'nunca' 'ofrecía' 'olor' 'opciones' 'ortografía'
 'papel' 'parecía' 'pedimos' 'personal' 'plaga' 'plato' 'platos' 'podía'
 'postre' 'precios' 'presentados' 'proporcionaron' 'rancio' 'reservas'
 'respetaron' 'restaurante' 'sabor' 'sazonados' 'ser' 'servicio' 'servida'
 'servilletas' 'sido' 'sillas' 'sucias' 'sucio' 'suficiente' 'tardó'
 'trajeron' 'utensilios' 'vegetarianas' 'vegetariano' 'veían' 'vino'
 'área']


In [6]:
# Y construimos un dataframe para mostrar el resultado: por cada documento las ocurrencias de cada token
pd.DataFrame(TFIDF.toarray(), columns=vocab)

,abarrotado,acondicionado,agrio,aire,alergias,alimentarias,almacenado,alta,atento,baño,...,sucio,suficiente,tardó,trajeron,utensilios,vegetarianas,vegetariano,veían,vino,área
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.415749,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.492489,0.000000,0.00000,0.492489,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.5,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.5
5,0.536162,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.415749,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0


## Implementación de LDA

In [7]:
# Creación de un objeto de clase LDA con sus componentes
lda_model = LatentDirichletAllocation(n_components = 3, max_iter = 20, random_state = 20)

# Extracción de los tópicos
X_topics = lda_model.fit_transform(TFIDF)

topic_words = lda_model.components_
topic_words

array([[0.33451763, 0.33447347, 0.77847526, 0.33447347, 0.33438676,
        0.33438676, 0.77847526, 0.74639283, 0.33466421, 0.3343944 ,
        0.82276382, 0.33447347, 0.80521907, 0.76431802, 0.33479153,
        1.07751319, 0.33452126, 0.86088602, 0.33437673, 0.74639283,
        0.77777765, 0.33479153, 0.85951727, 0.69699865, 0.33469415,
        0.33476896, 0.33445881, 0.33450489, 1.27749411, 1.46208147,
        0.33437673, 0.86945273, 0.33466421, 0.3343944 , 0.33452126,
        0.33450489, 0.33469418, 0.33445881, 0.33438676, 0.77777765,
        0.76630833, 1.08686101, 0.74639283, 0.76015949, 0.76574619,
        0.77777765, 0.33450489, 0.74639283, 0.82276382, 0.33438676,
        0.33469415, 0.33589615, 0.86945273, 0.3343944 , 0.72644534,
        1.78937098, 0.33450941, 0.33469418, 0.76431802, 0.7514824 ,
        0.74639283, 0.86088602, 0.85951727, 0.33476896, 0.33464187,
        0.86088602, 0.33451763, 0.33451763, 0.33560106, 0.80521907,
        0.80521907, 0.33437673, 0.33445881, 0.33

## Palabras en cada tópico

In [8]:
#  Número de palabras a extraer de cada tópico

n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    sorted_topic_dist = np.argsort(topic_dist)
    
    topic_words = np.array(vocab)[sorted_topic_dist]
    
    topic_words = topic_words[:-n_top_words:-1]
    print ("Tópico", str(i+1), topic_words)

Tópico 1 ['pedimos' 'factura' 'errores' 'mal']
Tópico 2 ['restaurante' 'vegetarianas' 'desagradable' 'olor']
Tópico 3 ['personal' 'restaurante' 'atento' 'grosero']


## Distribución de documentos por tópico

In [9]:
doc_topic = lda_model.transform(TFIDF)  

for n in range(doc_topic.shape[0]):
    topic_doc = doc_topic[n].argmax()
    print ("Documento", n+1, " -- Tópico:" ,topic_doc)

Documento 1  -- Tópico: 2
Documento 2  -- Tópico: 2
Documento 3  -- Tópico: 0
Documento 4  -- Tópico: 0
Documento 5  -- Tópico: 1
Documento 6  -- Tópico: 2
Documento 7  -- Tópico: 0
Documento 8  -- Tópico: 0
Documento 9  -- Tópico: 0
Documento 10  -- Tópico: 0
Documento 11  -- Tópico: 1
Documento 12  -- Tópico: 1
Documento 13  -- Tópico: 0
Documento 14  -- Tópico: 2
Documento 15  -- Tópico: 0
Documento 16  -- Tópico: 1
Documento 17  -- Tópico: 2
Documento 18  -- Tópico: 2
Documento 19  -- Tópico: 1
Documento 20  -- Tópico: 1
Documento 21  -- Tópico: 0
Documento 22  -- Tópico: 2
Documento 23  -- Tópico: 1
Documento 24  -- Tópico: 1
Documento 25  -- Tópico: 0


# Cómo visualizar los tópicos en el espacio

In [1]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=17504e5a4be71260f82722ee582396e2dde97b60d8c642320e13c40b644b1da4
  Stored in directory: /Users/irene/Library/Caches/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3513 sha256=d0874940d14cfb42b13ef63d987fe91e81dfe453c06ff31f2323d7078b1c018d
  Stored in directory: /Users/irene/Library/Caches/pip/wheels/9d/ff/2f/afe4cd56f47de147407705626517d68bea0f3b74eb1fb168e6
Successfully built fst-pso miniful


In [10]:
# Importa las bibliotecas necesarias
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import numpy as np


# Prepara los datos para PyLDAvis
panel = pyLDAvis.sklearn.prepare(lda_model, TFIDF, TFIDFVectorizer, mds='tsne')

# Visualiza el modelo LDA
pyLDAvis.display(panel)


ModuleNotFoundError: No module named 'pyLDAvis.sklearn'